LangChain Tool
```
Tool : llm에이전트가 수행할 수 있는 외부기능이나 api를 말함 (날씨정보나 웹검색, 계산기)
이유 : gpt는 최신정보에 접근할 수 없다
구조 : 이름 설명 실행함수로직
tool 호출 방식 : GPT-4 모델들은 OpenAI의 함수호출 기능을 통해 tool을 직접 호출할 수 있다
```
외부 API 호출 Tool 구현 (날씨API, 뉴스API)


In [2]:
%pip install langchain-core langchain-openai openai python-dotenv requests

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# OpenWeather api 가져오기
# https://home.openweathermap.org 회원가입 하고
# https://home.openweathermap.org/api_keys 들어가서 api key 가져오기


# Newsapi 가져오기
# https://newsapi.org/


# .env에다가 각각 OPEN_WEATHER_MAP, NEWSAPI_API_KEY로 키 저장하기

In [ ]:
# .env 로드
from dotenv import load_dotenv
import os
load_dotenv()

print(os.environ["OPEN_WEATHER_MAP"][:5])
print(os.environ["NEWSAPI_API_KEY"][:5])

6acec


KeyError: 'WEATHER_API_KEY'

In [ ]:
# 날씨정보 조회 함수
from pydantic import BaseModel, Field

# 입력 스키마
class WeatherInput(BaseModel):
    city : str = Field(description="날씨를 조회할 도시 이름(영문)")

def get_weather(city: str) -> str:
    import requests
    api_key = os.getenv("OPEN_WEATHER_MAP")
    url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q":city,
        "appid": api_key,
        "units": "metric",  # 섭씨온도
        "lang": "kr"  # 한국어
    }
    response = requests.get(url, params=params)
    data =  response.json()
    if data.get('cod') != 200:
        return "날씨 정보를 가져올 수 없습니다."
    temp = data['main']['temp']
    desc = data['weather'][0]['description']
    return f"{city}의 현재 기온은 {temp}도이며, 날씨는 {desc}입니다."

In [ ]:
# pydantic 모델을 사용하여 입력값 검증
class WeatherInput(BaseModel):
    city : str = Field(description="날씨를 조회할 도시 이름(영문)")

class MyClass:
    def __init__(self, ab: str):
        self.city = ab

MyClass(ab=100)  # 사용자가 의도한 문자열이 전달되지 않아도 python은 타입에러를 발생시키지 않음
result = WeatherInput( city=100)    # pydantic 모델은 타입에러를 발생시킴
result.city

NameError: name 'BaseModel' is not defined

In [ ]:
get_weather('seoul')

NameError: name 'BaseModel' is not defined

In [ ]:
class NewsInput(BaseModel):
    keyword: str = Field(description="뉴스를 조회할 키워드")
    
def get_news(keyword:str) -> str:
    import requests
    api_key = os.getenv("NEWSAPI_API_KEY")
    url = "https://newsapi.org/v2/everything"
    params = {
        "q": keyword,
        "apiKey": api_key,
        "language": "ko",
        "sortBy": "publishedAt",  #최신뉴스
        "pageSize": 1  # 최근 1건
    }
    response = requests.get(url, params=params)
    data = response.json()    
    articles = data.get('articles')
    if not articles:
        return "해당 키워드에 대한 뉴스가 없습니다."
    
    # 가장 첫번째 뉴스선택
    top_news = articles[0]
    title = top_news.get('title', '제목 없음')
    source = top_news.get('source', {}).get('name', '출처 없음')
    return f'{keyword}에 대한 최신 뉴스: "{title}" (출처: {source})'

NameError: name 'BaseModel' is not defined

In [ ]:
print(get_news('취업'))

LangChain Tool 객체로 전환
```
get_weather, get_news 함수를 Tool로 등록
```

In [ ]:
from langchain_core.runnables import RunnableLambda
# 날씨함수 runnable -> tool 변환
weather_runnable = RunnableLambda(lambda x: get_weather(x['city']))
weather_tool = weather_runnable.as_tool(
    name="get_weather",
    description="도시의 현재 날씨 정보를 반환합니다.",
    args_schema=WeatherInput
)  
# 뉴스함수 runnable -> tool 변환
news_runnable = RunnableLambda(lambda x: get_news(x['keyword']))
news_tool = news_runnable.as_tool(
    name="get_news",
    description="키워드에 대한 최신 뉴스 정보를 반환합니다.",
    args_schema=NewsInput
)
print(f'tool 이름:{weather_tool.name} 설명 : {weather_tool.description}')
print(f'tool 이름:{news_tool.name} 설명 : {news_tool.description}')

In [ ]:
# 툴없이 
from langchain_openai import ChatOpenAI
# 프롬프트 템플릿
from langchain_core.prompts import ChatPromptTemplate
# 출력 파서
from langchain_core.output_parsers import StrOutputParser
user_question = '서울 날씨 알려줘'
# 사용자 질문을 처리하는 프롬프트 템플릿
user_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 날씨와 뉴스 정보를 제공하는 AI입니다."),
    ("human", "{question}")
])
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
chain = user_prompt | llm | StrOutputParser()
# 사용자 질문을 처리하는 체인 실행
result = chain.invoke({"question": user_question})
print(result)

LLM 최종 답변 생성 요청

In [ ]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage
# 프롬프트구성 : 시스템메시지 + 사용자 질문 + 결과를 맥락으로 제공
weather_info = get_weather('seoul')
system_prompt = "당신은 유용한 AI 어시스턴트입니다. 사용자 질문에 맞게 제공된 정보를 활용해 답변하세요"
tool_info_prompt = f'도구가 제공한 추가 정보:{weather_info}'
ask_prompt = '위 정보를 참고해서 사용자 질문에 답변하세요'
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_question),
    AIMessage(content=tool_info_prompt),  #도구 결과를 마치 ai의 답변처럼 추가가
    HumanMessage(content=ask_prompt)
]
result = llm(messages)   # LLM에 메시지 전달
print(result.content)

여러 tool로 연결시켜서

In [ ]:
# 에이전트를 생성하고 등록
from langchain.agents import initialize_agent, AgentType
# 앞에서 만든 tool 객체 리스트를 준비
tools = [weather_tool, news_tool]
# openai 함수 기반 에이전트 생성
agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent_type = AgentType.OPENAI_FUNCTIONS,
    verbose = True # 에이전트의 동작을 출력하도록 설정
)